# Imports

In [ ]:
%cd '/opt/itmo-bigdata/hw2'
!pip install poetry
!poetry config virtualenvs.create false
!poetry install --no-root

In [14]:
import pandas as pd
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.linear_model import SGDRegressor
from sklearn.feature_extraction.text import TfidfVectorizer

# Block #1

## Loading files to Hadoop

In [ ]:
!apt install wget unzip -y

In [ ]:
%cd '/opt/itmo-bigdata/hw2/hadoop-notebook'
!rm -rf 'ml-latest-small'
!wget -q 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip' -O 'ml-latest-small.zip'
!unzip 'ml-latest-small.zip'
!rm 'ml-latest-small.zip'

In [22]:
!hdfs dfsadmin -safemode leave
!hdfs dfs -rm -r ml-latest-small
!hdfs dfs -put ml-latest-small

Safe mode is OFF
Deleted ml-latest-small


## Set up Spark

In [ ]:
%cd '/opt/itmo-bigdata/hw2/hadoop-notebook'
conf = SparkConf().set("spark.executor.instances", "2").set("spark.executor.cores", "1").set("spark.executor.memory", "1g")

In [ ]:
spark = SparkSession.builder.master("yarn").appName("kopecky_spark").getOrCreate()

In [27]:
spark

## ResourceManager & Spark UI Screens

### 'kopecky_spark' application in ResourceManager
![resource manager screen](images/rm_screen.png)

### 'kopecky_spark' application in Spark UI
![spark ui screen](images/spark_ui_screen.png)

### Executors in Spark UI
![spark ui screen](images/spark_ui_executors_screen.png)

## Spark Jobs

In [28]:
ratings_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("rating", DoubleType()),
    StructField("timestamp", LongType()),
])

ratings_df = (spark
              .read
              .format("csv")
              .option("header", "True")
              .schema(ratings_schema)
              .load("ml-latest-small/ratings.csv")
              )

ratings_df.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows



In [29]:
ratings_df

DataFrame[userId: int, movieId: int, rating: double, timestamp: bigint]

In [30]:
print(f'Number of rows in the Ratings dataset: {ratings_df.count()}')

Number of rows in the Ratings dataset: 100836


In [31]:
tags_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("tag", StringType()),
    StructField("timestamp", LongType()),
])

tags_df = (spark
           .read
           .format("csv")
           .option("header", "True")
           .schema(tags_schema)
           .load("ml-latest-small/tags.csv"))

tags_df.show(10)

+------+-------+-----------------+----------+
|userId|movieId|              tag| timestamp|
+------+-------+-----------------+----------+
|     2|  60756|            funny|1445714994|
|     2|  60756|  Highly quotable|1445714996|
|     2|  60756|     will ferrell|1445714992|
|     2|  89774|     Boxing story|1445715207|
|     2|  89774|              MMA|1445715200|
|     2|  89774|        Tom Hardy|1445715205|
|     2| 106782|            drugs|1445715054|
|     2| 106782|Leonardo DiCaprio|1445715051|
|     2| 106782|  Martin Scorsese|1445715056|
|     7|  48516|     way too long|1169687325|
+------+-------+-----------------+----------+
only showing top 10 rows



In [32]:
tags_df

DataFrame[userId: int, movieId: int, tag: string, timestamp: bigint]

In [33]:
print(f'Number of rows in the Tags dataset: {tags_df.count()}')

Number of rows in the Tags dataset: 3683


In [34]:
spark.stop()

## Spark Jobs Screens

### All jobs created as a result of two calls to the show and count methods
![block1 jobs](images/block1_jobs.png)

### All stages created as a result of two calls to the show and count methods
![block1 stages](images/block1_stages.png)

It is worth noting a rather interesting behavior when calling count(). For its execution two stages are really executed as expected, but it is done in an unusual way: two jobs are created, the first one receives data from the table, and the second job already has two stages, and the first stage is the same as the first job, so it is skipped and only the second stage is executed, which counts rows. After a little research I managed to find out that this is due to the fact that a high-level API is used, not RDD, because of which spark can apply non-obvious optimizations, including more than one job for an action.

### First count job example
![block1 count job1](images/block1_count_job1.png)

### Second count job example
![block1 count job2](images/block1_count_job2.png)

# Block #2

### Start app

In [ ]:
%cd '/opt/itmo-bigdata/hw2/hadoop-notebook'

In [ ]:
conf = SparkConf().set("spark.executor.instances", "2").set("spark.executor.cores", "1").set("spark.executor.memory", "1g")

spark = SparkSession.builder.master("yarn").appName("kopecky_spark").getOrCreate()

ratings_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("rating", DoubleType()),
    StructField("timestamp", LongType()),
])

ratings_df = (spark
              .read
              .format("csv")
              .option("header", "True")
              .schema(ratings_schema)
              .load("ml-latest-small/ratings.csv")
              )

tags_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("tag", StringType()),
    StructField("timestamp", LongType()),
])

tags_df = (spark
           .read
           .format("csv")
           .option("header", "True")
           .schema(tags_schema)
           .load("ml-latest-small/tags.csv"))

### Counting the number of unique movies and unique users in the "ratings" table

In [37]:
unique_movies = ratings_df.select("movieId").distinct().count()
print("Number of unique movies:", unique_movies)

unique_users = ratings_df.select("userId").distinct().count()
print("Number of unique users:", unique_users)

Number of unique movies: 9724


Number of unique users: 610


### Counting how many movie ratings were given a score of 4.0 or higher

In [38]:
high_ratings_count = ratings_df.filter(ratings_df['rating'] >= 4.0).count()
print("Number of ratings >= 4.0:", high_ratings_count)

Number of ratings >= 4.0: 48580


### Displaying the top 100 movies with the highest rating

In [39]:
top_100_movies = (ratings_df
                   .groupBy("movieId")
                   .agg(avg("rating").alias("average_rating"))
                   .orderBy(col("average_rating").desc())
                   .limit(100))
top_100_movies.show(100)

+-------+--------------+
|movieId|average_rating|
+-------+--------------+
|   2196|           5.0|
|  88448|           5.0|
| 172909|           5.0|
|  67618|           5.0|
|   3496|           5.0|
|    148|           5.0|
| 142444|           5.0|
|    496|           5.0|
|   8911|           5.0|
| 118834|           5.0|
| 173963|           5.0|
| 156025|           5.0|
|   5513|           5.0|
| 120130|           5.0|
|  26350|           5.0|
|   1349|           5.0|
| 147300|           5.0|
| 122092|           5.0|
|    633|           5.0|
|  71268|           5.0|
|    876|           5.0|
|  99636|           5.0|
|  53578|           5.0|
|   6086|           5.0|
| 160644|           5.0|
|  47736|           5.0|
|   5490|           5.0|
| 164367|           5.0|
| 172583|           5.0|
| 172589|           5.0|
| 126921|           5.0|
|   3473|           5.0|
|   3795|           5.0|
|  50999|           5.0|
|   3941|           5.0|
| 141718|           5.0|
|    626|           5.0|


### Calculating the time difference in seconds between the time a user tagged a movie and the time the same user rated the movie. Outputting the average time delta.

In [40]:
tags_df = tags_df.withColumnRenamed("timestamp", "tag_timestamp")
ratings_df = ratings_df.withColumnRenamed("timestamp", "rating_timestamp")

joined_df = tags_df.join(ratings_df, ["userId", "movieId"])
time_diff_df = joined_df.withColumn("time_diff", abs(col("tag_timestamp") - col("rating_timestamp")))
average_time_diff_df = time_diff_df.agg(avg("time_diff").alias("average_time_diff"))

average_time_diff_seconds = average_time_diff_df.collect()[0]["average_time_diff"]
print("Average time difference (in seconds):", average_time_diff_seconds)

Average time difference (in seconds): 29203715.568469506


### Calculating the average rating from each user, and then outputting the average of all these average ratings across all users.

In [41]:
average_ratings_per_user = ratings_df.groupBy("userId").agg(avg("rating").alias("average_rating"))
overall_average_rating = average_ratings_per_user.agg(avg("average_rating").alias('avg_rating'))
print("Overall average rating:", overall_average_rating.collect()[0]["avg_rating"])

Overall average rating: 3.6572223377474016


### Stop app

In [42]:
spark.stop()

# Block #3

### Start app

In [ ]:
%cd '/opt/itmo-bigdata/hw2/hadoop-notebook'

In [ ]:
conf = SparkConf().set("spark.executor.instances", "2").set("spark.executor.cores", "1").set("spark.executor.memory", "1g")

spark = SparkSession.builder.master("yarn").appName("kopecky_spark").getOrCreate()

ratings_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("rating", DoubleType()),
    StructField("timestamp", LongType()),
])

ratings_df = (spark
              .read
              .format("csv")
              .option("header", "True")
              .schema(ratings_schema)
              .load("ml-latest-small/ratings.csv")
              )

tags_schema = StructType(fields=[
    StructField("userId", IntegerType()),
    StructField("movieId", IntegerType()),
    StructField("tag", StringType()),
    StructField("timestamp", LongType()),
])

tags_df = (spark
           .read
           .format("csv")
           .option("header", "True")
           .schema(tags_schema)
           .load("ml-latest-small/tags.csv"))

### UDF

In [47]:
joined_df = tags_df.join(ratings_df, ["userId", "movieId"])
joined_pandas_df = joined_df.toPandas()

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(joined_pandas_df['tag'])
y = joined_pandas_df['rating']

sgd_regressor = SGDRegressor()
sgd_regressor.fit(X_tfidf, y)

def predict_rating(tag):
    transformed_tag = tfidf.transform([tag])
    return float(sgd_regressor.predict(transformed_tag)[0])

predict_rating_udf = udf(predict_rating, FloatType())

ratings_predictions = tags_df.withColumn("predicted_rating", predict_rating_udf(col("tag")))

ratings_predictions.show(50)

+------+-------+--------------------+----------+----------------+
|userId|movieId|                 tag| timestamp|predicted_rating|
+------+-------+--------------------+----------+----------------+
|     2|  60756|               funny|1445714994|        4.368023|
|     2|  60756|     Highly quotable|1445714996|       3.9453294|
|     2|  60756|        will ferrell|1445714992|        4.031532|
|     2|  89774|        Boxing story|1445715207|         3.93356|
|     2|  89774|                 MMA|1445715200|       3.4211416|
|     2|  89774|           Tom Hardy|1445715205|       3.8342075|
|     2| 106782|               drugs|1445715054|       4.2840347|
|     2| 106782|   Leonardo DiCaprio|1445715051|        4.236813|
|     2| 106782|     Martin Scorsese|1445715056|        3.849058|
|     7|  48516|        way too long|1169687325|       3.4277177|
|    18|    431|           Al Pacino|1462138765|       3.8518403|
|    18|    431|            gangster|1462138749|       3.6180396|
|    18|  

Stages
![block3 stages](images/block3_stages.png)

Diagrams

![block3 diagram 1](images/block3_udf_diagram1.png)
![block3 diagram 1](images/block3_udf_diagram2.png)

### RMSE

In [48]:
full_predictions = ratings_predictions.join(ratings_df, ["userId", "movieId"])
evaluator = RegressionEvaluator(labelCol="rating", predictionCol="predicted_rating", metricName="rmse")
rmse = evaluator.evaluate(full_predictions)
print("RMSE:", rmse)

RMSE: 0.8743434511207214


Stages
![block3 rmse stages](images/block3_rmse_stages.png)

Diagrams

![block3 dag 1](images/block3_dag_visualization1.png)
![block3 dag 1](images/block3_dag_visualization2.png)

### Stop app

In [49]:
spark.stop()